## MODEL KANDİLLİ DATASET

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import geopandas as gpd

# Load the data from the text file
file_path = '/content/1915-2023.txt'  # Replace with your file path

# Try different encodings if 'utf-8' does not work
try:
    kandilli_data = pd.read_csv(file_path, encoding='utf-8',sep="\t")
except UnicodeDecodeError:
    kandilli_data = pd.read_csv(file_path, encoding='latin1',sep="\t")  # or 'iso-8859-1' or 'cp1252'

kandilli_data.head()


,No,Deprem Kodu,Olus tarihi,Olus zamani,Enlem,Boylam,Der(km),xM,MD,ML,Mw,Ms,Mb,Tip,Yer
0,1,20230831083800,2023.08.31,08:38:00.25,36.0547,27.1385,7.7,3.7,0.0,3.6,3.7,0.0,0.0,Ke,AKDENIZ
1,2,20230829060305,2023.08.29,06:03:05.24,38.0215,38.3377,6.3,3.9,0.0,3.8,3.9,0.0,0.0,Ke,YESILYAYLA-CELIKHAN (ADIYAMAN) [South 3.2 km]
2,3,20230829045814,2023.08.29,04:58:14.13,38.9088,32.9598,3.5,5.1,0.0,5.1,4.9,0.0,0.0,Ke,TAVSANCALI-KULU (KONYA) [South West 5.1 km]
3,4,20230827030942,2023.08.27,03:09:42.97,36.3487,27.8080,65.3,4.4,0.0,4.4,0.0,0.0,0.0,Ke,AKDENIZ
4,5,20230826091713,2023.08.26,09:17:13.88,35.8478,35.9930,8.1,3.6,0.0,3.5,3.6,0.0,0.0,Ke,YAYLADAGI (HATAY) [South West 8.8 km]


In [ ]:
kandilli_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20174 entries, 0 to 20173
Data columns (total 15 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   No           20174 non-null  int64  
 1   Deprem Kodu  20174 non-null  int64  
 2   Olus tarihi  20174 non-null  object 
 3   Olus zamani  20174 non-null  object 
 4   Enlem        20174 non-null  float64
 5   Boylam       20174 non-null  float64
 6   Der(km)      20174 non-null  float64
 7   xM           20174 non-null  float64
 8   MD           20174 non-null  float64
 9   ML           20174 non-null  float64
 10  Mw           7193 non-null   float64
 11  Ms           20174 non-null  float64
 12  Mb           20174 non-null  float64
 13  Tip          20174 non-null  object 
 14  Yer          20174 non-null  object 
dtypes: float64(9), int64(2), object(4)
memory usage: 2.3+ MB


In [ ]:
# Gereksiz sütunları çıkar
kandilli_data.drop(['Deprem Kodu','Tip'], axis=1, inplace=True)

In [ ]:
kandilli_data['Mw'].fillna(kandilli_data['Mw'].mean(), inplace=True)
kandilli_data['Mw'].info()

<class 'pandas.core.series.Series'>
RangeIndex: 20174 entries, 0 to 20173
Series name: Mw
Non-Null Count  Dtype  
--------------  -----  
20174 non-null  float64
dtypes: float64(1)
memory usage: 157.7 KB


# Tarih Zaman tahmin eden Kötü Model

In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense
from sklearn.preprocessing import MinMaxScaler
import numpy as np

# Veri setini yükleme ve ön işleme
kandilli_data['TarihZaman'] = pd.to_datetime(kandilli_data['Olus tarihi'] + ' ' + kandilli_data['Olus zamani'])
kandilli_data.sort_values('TarihZaman', inplace=True)

# Özellikler ve hedef değişken
features = kandilli_data[['Enlem', 'Boylam']]
target = kandilli_data['TarihZaman']

# Özelliklerin ve hedefin ölçeklendirilmesi
scaler = MinMaxScaler()
features_scaled = scaler.fit_transform(features)
target_scaled = scaler.fit_transform(target.values.reshape(-1, 1))

# Veriyi eğitim ve test setlerine ayırma
X_train, X_test, y_train, y_test = train_test_split(features_scaled, target_scaled, test_size=0.2, random_state=42)

# LSTM modelinin oluşturulması
model = Sequential()
model.add(LSTM(50, activation='relu', input_shape=(1, 2)))
model.add(Dense(1))
model.compile(optimizer='adam', loss='mean_squared_error')

# Modeli eğitme
X_train = X_train.reshape((X_train.shape[0], 1, X_train.shape[1]))

model.fit(X_train, y_train, epochs=100, batch_size=32)

# Tahmin ve değerlendirme (Örnek olarak)
from sklearn.metrics import mean_absolute_error, mean_squared_error
import numpy as np

# Test verilerini model için uygun formata dönüştürme
X_test_reshaped = X_test.reshape((X_test.shape[0], 1, X_test.shape[1]))

# Test verileri üzerinde tahmin yapma
predictions = model.predict(X_test_reshaped)

# Gerçek değerlerin ölçeklendirmesini geri alma
y_test_rescaled = scaler.inverse_transform(y_test)
predictions_rescaled = scaler.inverse_transform(predictions)

# Performans metriklerini hesaplama
mae = mean_absolute_error(y_test_rescaled, predictions_rescaled)
rmse = np.sqrt(mean_squared_error(y_test_rescaled, predictions_rescaled))

print("Mean Absolute Error (MAE):", mae)
print("Root Mean Squared Error (RMSE):", rmse)

Epoch 1/100
505/505 [==============================] - 29s 2ms/step - loss: 0.1284
Epoch 2/100
505/505 [==============================] - 1s 2ms/step - loss: 0.0352
Epoch 3/100
505/505 [==============================] - 1s 2ms/step - loss: 0.0336
Epoch 4/100
505/505 [==============================] - 1s 2ms/step - loss: 0.0334
Epoch 5/100
505/505 [==============================] - 1s 2ms/step - loss: 0.0333
Epoch 6/100
505/505 [==============================] - 1s 2ms/step - loss: 0.0331
Epoch 7/100
505/505 [==============================] - 1s 2ms/step - loss: 0.0328
Epoch 8/100
505/505 [==============================] - 1s 3ms/step - loss: 0.0326
Epoch 9/100
505/505 [==============================] - 1s 2ms/step - loss: 0.0326
Epoch 10/100
505/505 [==============================] - 1s 2ms/step - loss: 0.0326
Epoch 11/100
505/505 [==============================] - 1s 2ms/step - loss: 0.0325
Epoch 12/100
505/505 [==============================] - 1s 2ms/step - loss: 0.0325
Epoch 13/100

# **Tarih  Zaman tahmin eden İyi Model**

In [ ]:
from sklearn.model_selection import TimeSeriesSplit
from sklearn.metrics import mean_absolute_error, mean_squared_error
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense
import numpy as np

# Model oluşturma fonksiyonu
def create_model(X_train, y_train, X_val, y_val):
    model = Sequential()
    model.add(LSTM(50, activation='relu', input_shape=(1, X_train.shape[2])))
    model.add(Dense(1))
    model.compile(optimizer='adam', loss='mean_squared_error')
    model.fit(X_train, y_train, epochs=100, batch_size=32, verbose=0, validation_data=(X_val, y_val))
    return model

# TimeSeriesSplit ile çapraz doğrulama
tscv = TimeSeriesSplit(n_splits=5)
mae_scores = []
rmse_scores = []

for train_index, test_index in tscv.split(features_scaled):
    X_train, X_test = features_scaled[train_index], features_scaled[test_index]
    y_train, y_test = target_scaled[train_index], target_scaled[test_index]

    # Veriyi LSTM için uygun formata dönüştürme
    X_train = X_train.reshape((X_train.shape[0], 1, X_train.shape[1]))
    X_test = X_test.reshape((X_test.shape[0], 1, X_test.shape[1]))

    # Modeli oluştur ve eğit
    model = create_model(X_train, y_train, X_test, y_test)

    # Tahmin ve değerlendirme
    predictions = model.predict(X_test)
    mae = mean_absolute_error(y_test, predictions)
    rmse = np.sqrt(mean_squared_error(y_test, predictions))

    mae_scores.append(mae)
    rmse_scores.append(rmse)

# Ortalama skorları yazdır
print("Ortalama MAE:", np.mean(mae_scores))
print("Ortalama RMSE:", np.mean(rmse_scores))

106/106 [==============================] - 0s 1ms/step
Ortalama MAE: 0.21717607336481276
Ortalama RMSE: 0.22322668395834847


In [ ]:
# Tarih ve saat sütunlarını işleme
kandilli_data['Year'] = pd.to_datetime(kandilli_data['Olus tarihi']).dt.year
kandilli_data['Month'] = pd.to_datetime(kandilli_data['Olus tarihi']).dt.month
kandilli_data['Day'] = pd.to_datetime(kandilli_data['Olus tarihi']).dt.day
kandilli_data['Hour'] = pd.to_datetime(kandilli_data['Olus zamani']).dt.hour
kandilli_data['Minute'] = pd.to_datetime(kandilli_data['Olus zamani']).dt.minute
kandilli_data.drop(['Olus tarihi', 'Olus zamani'], axis=1, inplace=True)

In [ ]:
kandilli_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20174 entries, 0 to 20173
Data columns (total 16 columns):
 #   Column   Non-Null Count  Dtype  
---  ------   --------------  -----  
 0   No       20174 non-null  int64  
 1   Enlem    20174 non-null  float64
 2   Boylam   20174 non-null  float64
 3   Der(km)  20174 non-null  float64
 4   xM       20174 non-null  float64
 5   MD       20174 non-null  float64
 6   ML       20174 non-null  float64
 7   Mw       20174 non-null  float64
 8   Ms       20174 non-null  float64
 9   Mb       20174 non-null  float64
 10  Yer      20174 non-null  object 
 11  Year     20174 non-null  int64  
 12  Month    20174 non-null  int64  
 13  Day      20174 non-null  int64  
 14  Hour     20174 non-null  int64  
 15  Minute   20174 non-null  int64  
dtypes: float64(9), int64(6), object(1)
memory usage: 2.5+ MB


In [ ]:
# Kategorik sütunları one-hot encoding ile dönüştürme
kandilli_data = pd.get_dummies(kandilli_data, columns=['Yer'])

In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import confusion_matrix, classification_report, accuracy_score
from tensorflow.keras.utils import to_categorical

# Veri setinizi yükleyin
# Örnek: kandilli_data = pd.read_csv('veri_seti.csv')

# "Mw" değerlerini kategorilere ayırma
bins = [-np.inf, 3.0, 5.0, np.inf]  # Örnek eşik değerleri: 0-3 düşük, 3-5 orta, 5+ yüksek
labels = [0, 1, 2]  # 0: düşük, 1: orta, 2: yüksek
kandilli_data['Mw_category'] = pd.cut(kandilli_data['Mw'], bins=bins, labels=labels)
kandilli_data['Mw_category'].info()


<class 'pandas.core.series.Series'>
RangeIndex: 20174 entries, 0 to 20173
Series name: Mw_category
Non-Null Count  Dtype   
--------------  -----   
20174 non-null  category
dtypes: category(1)
memory usage: 20.0 KB


In [ ]:


# Özellikler ve hedef değişken
features = kandilli_data.drop(['Mw', 'Mw_category'], axis=1)
target = kandilli_data['Mw_category']

# Veriyi ölçeklendirme
scaler = StandardScaler()
features_scaled = scaler.fit_transform(features)

# Veriyi eğitim ve test setlerine ayırma
X_train, X_test, y_train, y_test = train_test_split(features_scaled, target, test_size=0.2, random_state=42)

# Sınıflandırma modelini eğitme
model = RandomForestClassifier(random_state=42)
model.fit(X_train, y_train)

# Tahminler ve model değerlendirme
predictions = model.predict(X_test)
cm = confusion_matrix(y_test, predictions)
acc = accuracy_score(y_test, predictions)
report = classification_report(y_test, predictions)

print("Confusion Matrix:\n", cm)
print("\nAccuracy Score:", acc)
print("\nClassification Report:\n", report)

Confusion Matrix:
 [[  44   95    1]
 [  13 3727    8]
 [   0   14  133]]

Accuracy Score: 0.9675340768277572

Classification Report:
               precision    recall  f1-score   support

           0       0.77      0.31      0.45       140
           1       0.97      0.99      0.98      3748
           2       0.94      0.90      0.92       147

    accuracy                           0.97      4035
   macro avg       0.89      0.74      0.78      4035
weighted avg       0.96      0.97      0.96      4035



#İyi Model

In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout

kandilli_data['Tarih'] = pd.to_datetime(kandilli_data['Olus tarihi'] + ' ' + kandilli_data['Olus zamani'])
kandilli_data.sort_values('Tarih', inplace=True)

features = kandilli_data[['Enlem', 'Boylam', 'Der(km)']]  # Örnek özellikler
kandilli_data['Mw'].fillna(kandilli_data['Mw'].mean(), inplace=True)

target = kandilli_data['Mw']


scaler = MinMaxScaler()
features_scaled = scaler.fit_transform(features)
target_scaled = scaler.fit_transform(target.values.reshape(-1,1))
X_train, X_test, y_train, y_test = train_test_split(features_scaled, target_scaled, test_size=0.2, random_state=42)
X_train = X_train.reshape((X_train.shape[0], 1, X_train.shape[1]))
X_test = X_test.reshape((X_test.shape[0], 1, X_test.shape[1]))
model = Sequential()
model.add(LSTM(50, activation='relu', input_shape=(1, features_scaled.shape[1])))
model.add(Dropout(0.2))
model.add(Dense(1))

model.compile(optimizer='adam', loss='mean_squared_error')
model.fit(X_train, y_train, epochs=100, batch_size=32, validation_data=(X_test, y_test), verbose=1)
loss = model.evaluate(X_test, y_test, verbose=0)
print(f'Test Loss: {loss}')
predictions = model.predict(X_test)



Epoch 1/100
505/505 [==============================] - 2s 2ms/step - loss: 0.0356 - val_loss: 0.0116
Epoch 2/100
505/505 [==============================] - 1s 2ms/step - loss: 0.0127 - val_loss: 0.0112
Epoch 3/100
505/505 [==============================] - 1s 3ms/step - loss: 0.0126 - val_loss: 0.0112
Epoch 4/100
505/505 [==============================] - 1s 2ms/step - loss: 0.0123 - val_loss: 0.0112
Epoch 5/100
505/505 [==============================] - 1s 2ms/step - loss: 0.0119 - val_loss: 0.0112
Epoch 6/100
505/505 [==============================] - 1s 3ms/step - loss: 0.0118 - val_loss: 0.0111
Epoch 7/100
505/505 [==============================] - 1s 2ms/step - loss: 0.0115 - val_loss: 0.0111
Epoch 8/100
505/505 [==============================] - 1s 2ms/step - loss: 0.0114 - val_loss: 0.0111
Epoch 9/100
505/505 [==============================] - 1s 2ms/step - loss: 0.0114 - val_loss: 0.0111
Epoch 10/100
505/505 [==============================] - 1s 2ms/step - loss: 0.0112 - val_lo

In [ ]:
 #sonn
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.neural_network import MLPRegressor
from sklearn.metrics import mean_squared_error
import numpy as np
 # Gereksiz sütunları çıkar
kandilli_data.drop(['xM', 'MD', 'ML', 'Ms', 'Mb'], axis=1, inplace=True)
# Handle missing values in 'Mw' column by replacing them with the mean
kandilli_data['Mw'].fillna(kandilli_data['Mw'].mean(), inplace=True)


# Veri setini özellikler ve hedef değişken olarak bölme
features = kandilli_data.drop('Mw', axis=1)
target = kandilli_data['Mw']

# Veri setini eğitim ve test setlerine bölme
X_train, X_test, y_train, y_test = train_test_split(features, target, test_size=0.2, random_state=42)

# Özellikleri ölçeklendirme
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Yapay Sinir Ağı modelini oluşturma ve eğitme
model = MLPRegressor(hidden_layer_sizes=(100,), activation='relu', random_state=42, max_iter=500)
model.fit(X_train_scaled, y_train)

# Test verileri üzerinde tahmin yapma ve modelin performansını değerlendirme
predictions = model.predict(X_test_scaled)
rmse = np.sqrt(mean_squared_error(y_test, predictions))

rmse

2.0344778017394614

# KÖTÜ MODEL

In [ ]:
# Model için girdi ve hedef sütunları ayırma
X = kandilli_data.drop('xM', axis=1)
y = kandilli_data['xM']

# Verileri ölçeklendirme
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# Model oluşturma fonksiyonu
def create_model():
    model = Sequential()
    model.add(Dense(64, activation='relu', input_dim=X_scaled.shape[1]))
    model.add(Dropout(0.2))
    model.add(Dense(32, activation='relu'))
    model.add(Dense(1))
    model.compile(optimizer='adam', loss='mean_squared_error')
    return model

# K-Fold Cross Validation
kf = KFold(n_splits=5, shuffle=True, random_state=42)
mse_scores = []
r2_scores = []

for train_index, test_index in kf.split(X_scaled):
    X_train, X_test = X_scaled[train_index], X_scaled[test_index]
    y_train, y_test = y.iloc[train_index], y.iloc[test_index]

    model = create_model()
    model.fit(X_train, y_train, epochs=10, batch_size=32, verbose=0)

    predictions = model.predict(X_test)
    mse = mean_squared_error(y_test, predictions)
    r2 = r2_score(y_test, predictions)

    mse_scores.append(mse)
    r2_scores.append(r2)

# Sonuçların ortalamasını hesaplama
avg_mse = np.mean(mse_scores)
avg_r2 = np.mean(r2_scores)

print("Ortalama MSE:", avg_mse)
print("Ortalama R^2 Score:", avg_r2)

127/127 [==============================] - 0s 2ms/step
Ortalama MSE: 0.6661339827035793
Ortalama R^2 Score: -1.3244020482562764
